Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('./SD-Datasets')

Mounted at ./SD-Datasets


In [2]:
train_dir = './SD-Datasets/MyDrive/SD-Datasets/Skin Disease_dataset/TRAIN'
test_dir = './SD-Datasets/MyDrive/SD-Datasets/Skin Disease_dataset/TEST'

Augmentasi Data

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Image size
img_size = (224, 224)

# Training with more augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Train and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 512 images belonging to 8 classes.
Found 128 images belonging to 8 classes.


In [7]:
# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

Found 160 images belonging to 8 classes.


In [8]:
# Load InceptionV3 with pretrained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze the base model (more layers)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

87910968/87910968 [==============================] - 1s 0us/step


In [9]:
# Compile the model
opt = Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [10]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[LearningRateScheduler(lr_scheduler), early_stopping]
)

Epoch 1/50
16/16 [==============================] - 266s 14s/step - loss: 1.9714 - accuracy: 0.2363 - val_loss: 1.9191 - val_accuracy: 0.2656 - lr: 1.0000e-04
Epoch 2/50
16/16 [==============================] - 15s 956ms/step - loss: 1.5158 - accuracy: 0.4688 - val_loss: 1.8438 - val_accuracy: 0.3438 - lr: 1.0000e-04
Epoch 3/50
16/16 [==============================] - 15s 940ms/step - loss: 1.0823 - accuracy: 0.6211 - val_loss: 1.6798 - val_accuracy: 0.4219 - lr: 1.0000e-04
Epoch 4/50
16/16 [==============================] - 15s 930ms/step - loss: 0.7934 - accuracy: 0.7344 - val_loss: 1.3589 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 5/50
16/16 [==============================] - 15s 927ms/step - loss: 0.6654 - accuracy: 0.7676 - val_loss: 1.3305 - val_accuracy: 0.5703 - lr: 1.0000e-04
Epoch 6/50
16/16 [==============================] - 15s 928ms/step - loss: 0.4986 - accuracy: 0.8340 - val_loss: 1.2835 - val_accuracy: 0.6094 - lr: 1.0000e-04
Epoch 7/50
16/16 [=======================

In [11]:
# Evaluation
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

5/5 [==============================] - 43s 11s/step - loss: 1.7782 - accuracy: 0.6313
Test Accuracy: 63.13%


In [12]:
# Classification Report
from sklearn.metrics import classification_report
import numpy as np

# Predict classes for test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# True classes for test set
true_classes = test_generator.classes

# Class labels
class_labels = list(test_generator.class_indices.keys())

# Generate classification report
report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

5/5 [==============================] - 4s 343ms/step
                  precision    recall  f1-score   support

           AKIEC       0.07      0.05      0.06        20
         ANGIOMA       0.05      0.05      0.05        20
      Basal-Cell       0.13      0.10      0.11        20
Benign-Keratosis       0.11      0.10      0.10        20
  Dermatofibroma       0.08      0.10      0.09        20
        MELANOMA       0.11      0.10      0.11        20
Melanocytic-Nevi       0.12      0.15      0.13        20
        Ringworm       0.10      0.10      0.10        20

        accuracy                           0.09       160
       macro avg       0.09      0.09      0.09       160
    weighted avg       0.09      0.09      0.09       160

